In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from __future__ import print_function
import torch.nn as nn
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn.functional as F

params = {
    "nc": 3,
    "nz": 100,
    "ngf": 64
}

In [9]:
class Generator(nn.Module):
    def __init__(self, params):
        super().__init__()
        self.tconv1 = nn.ConvTranspose2d(params['nz'], params["ngf"] * 8, kernel_size=4, stride=1, padding=0,
                                       bias=False)
        self.bn1 = nn.BatchNorm2d(params["ngf"]*8)
        
        self.tconv2 = nn.ConvTranspose2d(params["ngf"]*8, params["ngf"]*4,
                                       4, 2, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(params["ngf"]*4)
        
        self.tconv3 = nn.ConvTranspose2d(params["ngf"]*4, params["ngf"]*2,
                                       4, 2, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(params["ngf"]*2)
        
        self.tconv4 = nn.ConvTranspose2d(params["ngf"]*2, params["ngf"],
                                       4, 2, 1, bias=False)
        self.bn4 = nn.BatchNorm2d(params["ngf"])
        
        self.tconv5 = nn.ConvTranspose2d(params["ngf"], params["nc"], 4, 2, 1, bias=False)
        
    def forward(self, x):
        x = F.relu(self.bn1(self.tconv1(x)))
        x = F.relu(self.bn2(self.tconv2(x)))
        x = F.relu(self.bn3(self.tconv3(x)))
        x = F.relu(self.bn4(self.tconv4(x)))
        x = F.tanh(self.tconv5(x))
        return x

In [12]:
model = "../input/gan-model/COTS_GAN_v1.pth"
gen = Generator(params).cuda()
model = torch.load(model)
gen.load_state_dict(model['generator'])
gen.eval()

In [15]:
from tqdm import tqdm_notebook as tqdm

bz = 40
n_images = 40
plt.rcParams['figure.figsize'] = (4*n_images, n_images)
device = torch.device("cuda")
plt.ion()

for i in tqdm(range(0, n_images, bz)):
    gen_z = torch.randn(bz, params['nz'], 1, 1, device=device)
    gen_images = gen(gen_z)
    grid = torchvision.utils.make_grid(gen_images, nrow=4, padding=10, normalize=True)
    cpu_grid = grid.cpu()
    cpu_perm = np.transpose(cpu_grid, (1,2,0))
    plt.imshow(cpu_perm)
    
    torchvision.utils.save_image(gen_images.data, f"./gan_{i}.jpg")